## SD210 - Challenge
## TURQUETIL Antoine
## Récaputulatif de la méthode

Tout le travail a été réalisé avec scikit-learn sous python.

### I. Analyse des variables explicatives

La première chose à faire était de regarder plus ou moins en détail quelles variables étaient inutiles. En particulier certaines variables comme 'cited_nmiss' ou 'cited_age_std' ont été rejetées comme manifestement trop peu explicatives. 'cited_nmiss' ne prend en particulier qu'une valeur et est absente d'environ 60% des données. La plupart des variables ont toutefois été gardées même si certaines étaient doûteuses, dans l'espoir de gagner quelque points.

Une approche de type PCA a été tentée pour tenter de réduire la dimension du problème, mais on perdait trop en performances. De même pour l'évaluation d'importance des features avec les random forests.

L'analyse met aussi au jour la quantité importante de "trous" dans les données. On pourrait tenter d'inférer les données manquantes à partir des autres pour obtenir un dataset de meilleure qualité et augmenter la fiabilité de la de la prédiction. Cette idée n'a pas abouti pour le résultat final.

### II. Distinction catégorie/continuité

Comme c'était conseillé dans starting kit, on s'est assez vite penché sur la nature hétérogène des features : certaines sont catégorielles quand les autres sont continues. Il conviendra donc de les traiter différemment.

En premier lieu, on a cherché à entraîner un modèle pour les catégories et un autre pour les continues, avant de les "recoller". Régression logistique (polynomiale), KNN, SVM,... Beaucoup de modèles ont été expérimentés.

La partie critique de cette manière d'aborder le problème est le recollement :

- Le modèle 1 donne Y1 (vecteur à valeurs de 0 à 1 contenant les prédictions en probabilité) à partir de X1.
- Le modèle 2 donne Y2 à partir de X2.
- Le modèle 3 donne Y3 (réponse finale) à partir de Y1 et Y2.

Le modèle 3 est donc un problème de classification à deux variables. On a choisi initialement d'utiliser une régression logistique polynomiale.
Un exemple de prédictions à combiner :

<img src="recol.png">

Il se trouve que cette méthode n'a pas donné un score aussi élevé que d'autres méthodes plus simples à mettre en oeuvre. C'est à dire utiliser RandomForestClassifier.

### III. Les forêts aléatoires

Ensuite, on a essayé les forêts aléatoires. L'avantage de ce modèle est de pouvoir gérer simultanément des variables continues et catégorielles. Toutefois, si l'algorithme peut directement s'appliquer aux données continues, la question se pose pour les données catégorielles.

Pour des raisons évidentes de performance, il est tout d'abord nécessaire d'encoder les différentes valeurs prises par une variable catégorielles par des nombres (utilisation en particulier de LabelEncoder de sklearn).

En effet, les arbres de décisions fonctionnent en split, c'est à dire qu'ils séparent successivement les données sur la base d'un ordre total. Or, les données catégorielles sous maintenant sous forme numérique mais aucun ordre total ne peut être induit dessus, ce qui risque donc de tromper l'algorithme. Toutefois, de nombreuses sources sur le net semblent signaler que RandomForestClassifier est capable d'apprendre même sans one hot encoding, d'autres soutiennent le contraire... On a finalement gardé le one hot encoding pour les variables catégorielles.

Pour régler les paramètres (le plus important étant le nombre maximal de feuilles par arbre), on a d'abord utilisé la validation croisée. Quand il s'est avéré que le nombre de feuilles optimal était très élevé (supérieur à 10000), on a préféré passer à une validation plus simple en coupant le training set en deux parties.

A la fin, il fallait faire pousser plusieurs milliers d'arbres pour que le modèle converge, ce qui demande un certain temps, surtout quand il faut tester beaucoup de valeurs possibles pour 'max_leaf_node'. Pire encore que le temps, on a commencé à manquer de mémoire RAM, ce qui en général résulte en un plantage du kernel du notebook ipython.
Heureusement, pour combiner les prédictions de plusieurs forêts contenant le même nombre d'arbres, il suffit de faire la moyenne des prédictions de chaque forêt.
On peut donc faire pousser une forêt, prédire puis oublier le modèle pour libérer de la RAM, avant de faire pousser la forêt suivante.

Ce procédé permet de s'en sortir avec une seule machine, mais cela reste long. On a donc décidé de se servir des machines de l'école en recombinant les forêts aléatoires sur le même principe. Pour automatiser tout çela on s'est servi de scripts shell assez basiques lançant des scripts python via ssh, un autre script étant chargé de récupérer puis recombiner les prédictions produites par chaque machine.

### IV. Conclusion

Comment améliorer ce score ?

Peut-être qu'utiliser d'autres méthodes d'ensemble comme des arbres boostés aurait pu être judicieux. Malheureusement les performances de ces modèles sur sklearn ne sont pas aussi bonnes que pour les forêts aléatoires. D'autres libraires comme Smile pour Scala sont peut-être meilleures pour ce genre de modèles difficilement parallèlisables.

Une grande partie de la qualité de la prédiction tient dans le preprocessing : utiliser une méthode d'inférence pour les trous qui soit meilleure que Imputer est probablement la clé pour arriver aux meilleurs scores sur ce challenge.